In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

from langchain_groq import ChatGroq
groq_api_key = os.getenv("GRQO_API_KEY")
llm = ChatGroq(groq_api_key= groq_api_key, model="Llamma3.2-8b-8192")



In [5]:
from langchain.schema import AIMessage, HumanMessage, SystemMessage



In [3]:
speech = """ The speech of Imran khan ex Pakistani prime minister in United nations is one of the famous speech of Pkaistani Prime minister.
He spot light on four different but ignored topics like Kashmir issue, Islam od Phobia, Climate Change, Palestine Issue. 

"""

In [6]:
chat_message = [
    SystemMessage(content="You are expirt with experties in Summarization of text"),
    HumanMessage(content=f"Please provide the short and concise summary of the fpllowing speech./n Text: {speech}")
]

In [7]:
llm.get_num_tokens(speech)

NameError: name 'llm' is not defined

In [8]:
llm(chat_message).content

NameError: name 'llm' is not defined

In [ ]:
llm(chat_message).content

In [ ]:
from langchain.chains import LLMChain
from langchain import PromptTemplate

genericTemplate = """
write a summary of the following speech:
Speech:{speech}
Translate the precise summary to {language}
"""

prompt = PromptTemplate(
    input_variables= ["speech", "language"],
    template = genericTemplate
)
prompt

In [ ]:
Complete_prompt= prompt.format(speech= speech, language="French")
Complete_prompt

In [ ]:
llm.get_num_tokens(Complete_prompt)


In [ ]:
llm_chain = LLMChain(llm= llm, prompt= prompt)
summary = llm_chain.run({"speech": speech, "language": "French"})
summary

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("Attention.pdf")
docs = loader.load_and_split()
docs



In [ ]:
template = """
write a concise and short summary of the following speech:
Speech: {text}
"""
prompt = PromptTemplate(
    input_variable= ["speech"],
    template = template
)
prompt


In [ ]:
from langchain.chains.summarize import load_summarize_chain
chain = load_summarize_chain(llm, chain_type="stuff", verbose=True )
output_summary = chain.run(docs)


Map Reduced to Summarize large Documents

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
Splitter = RecursiveCharacterTextSplitter(chunk_size= 500, chunk_overlap= 50)
final_documents= Splitter.split_documents(docs) 
len(final_documents)


In [ ]:
chunks_prompt = """
Please Summarize the below speech:
Speech: {text}
Summary:
"""
map_prompt_template = PromptTemplate(

    input_variable = ["text"]
    template= chunks_prompt
)
map_prompt_template


In [ ]:
summary_chain = load_summarize_chain(
    llm=llm,
    chain_type="map_reduce",
    map_prompt = map_prompt_template 
)

In [ ]:
final_prompt = """
Provide the final summary of the of the entire speech with these important points.
add a Motivational Title, start a precise summary with an introduction and provide the summary 
in number of points for the speech.
Speech: {text}  
"""
final_prompt_template = PromptTemplate(
    input_variable= ["text"],
    template = final_template
)


In [ ]:
summary_chain = load_summarize_chain(
    llm=llm,
    chain_type="map_reduce",
    map_prompt= map_prompt_template,
    combine_prompt= final_prompt_template,
    verbose=True
)
summary_chain.run(final_documents)